In [0]:
import builtins
import sys
import uuid
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *

from pyspark.sql.functions import (
    col,                    
    when,                   
    lit,                    
    trim,                   
    upper,                 
    lower,                         
    coalesce,              
    concat,                            
    substring,             
    length,                
    cast,                  
    round,                
    sum,                   
    count,                 
    avg,                   
    max,                   
    min,                   
    countDistinct,         
    current_timestamp,     
    year,                  
    month,                 
    datediff,              
    to_date,               
    date_format,           
    rank,                  
    dense_rank,            
    lag,                  
    lead,
    decode,
    encode ,
    regexp_replace,
    lpad, 
    translate,
    collect_list,
    explode,
    split, 
    concat_ws,
    percentile_approx,
    xxhash64             
)

from pyspark.sql.types import (
    StringType,
    IntegerType,
    DoubleType,
    DateType,
    TimestampType,
    DecimalType,
)

from pyspark.sql import Window, DataFrame

from typing import Dict, Any

apply_storage_config(spark)

In [0]:
# Localizações dos dados da BRONZE
path_storage_bronze = f"{bronze_path}"
path_storage_silver = f"{silver_path}"
dbutils.fs.ls(bronze_path)

# Funções utilitárias do Projeto

In [0]:
def latin1_to_utf8(df: DataFrame, columns: list[str]) -> DataFrame:
    """ Tenta converter colunas do tipo StringType de iso-8859-1(latin1) para UTF-8 e retorna o dataframe com as colunas convertidas

        Parâmetros:
        df(DataFrame): Objeto DataFrame com os dados a serem convertidos para UTF-8
        columns(list[str]): Lista com os nomes das colunas a serem convertidas

        Retorna:
        DataFrame: DataFrame com as colunas convertidas para UTF-8
    """
    for column_name in columns:
        df = df.withColumn(
            column_name, 
            decode(encode(col(column_name), "ISO-8859-1"), "UTF-8")
        )

    return df


def strip_df(df: DataFrame, columns: list[str]) -> DataFrame:
    """
    Remove espaços em branco do início e do fim das strings nas colunas especificadas de um DataFrame Spark.

    Parâmetros:
        df (DataFrame): DataFrame de entrada.
        columns (list[str]): Lista com os nomes das colunas a serem tratadas.

    Retorna:
        DataFrame: DataFrame com as colunas especificadas tratadas com trim.
    """
    for column_name in columns:
        df = df.withColumn(column_name, trim(column_name))
    
    return df

# Mapeia e remove acentos.
def remove_accents(df: DataFrame, columns: list[str]) -> DataFrame:
  
    # Mapeamento de acentos
    accented = "áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ"
    unaccented = "aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC"

    for column_name in columns:
        df = df.withColumn(
            column_name,
            regexp_replace(
                translate(col(column_name), accented, unaccented),
                r"[^a-zA-Z0-9\s]",
                ""
            )
        )

    return df


#Converte colunas de data para o formato DateType (YYYY-MM-DD).   
def cast_dates(df: DataFrame, columns: list[str], input_format: str = "yyyyMMdd") -> DataFrame:
    """
    Converte colunas de string para o tipo DateType em um DataFrame Spark, utilizando o formato de data especificado.

    Parâmetros:
        df (DataFrame): DataFrame de entrada.
        columns (list[str]): Lista com os nomes das colunas a serem convertidas para data.
        input_format (str, opcional): Formato da data de entrada (padrão: "yyyyMMdd").

    Retorna:
        DataFrame: DataFrame com as colunas especificadas convertidas para DateType.
    """
    for column_name in columns:
        df = df.withColumn(
            column_name,
            to_date(col(column_name).cast("string"), input_format)
        )

    return df


def normalize_blanks_to_null(df: DataFrame, columns: list) -> DataFrame:
    """
    Converte para null:
      - valores já nulos (None/null)
      - strings vazias ("")
      - strings contendo apenas espaços ("   ", "\t", etc. após trim)
    
    Aplica somente em colunas string.
    Se 'columns' for None, aplica a todas as colunas string do DataFrame.
    Se 'columns' contiver colunas não-string, elas serão ignoradas.
    """
    # Define colunas alvo
    if columns is None:
        target_cols = [c for c, t in df.dtypes if t == "string"]
    else:
        # Usa apenas colunas que existem e são string
        string_cols = {c for c, t in df.dtypes if t == "string"}
        target_cols = [c for c in columns if c in string_cols]

    # Aplica a normalização
    for c in target_cols:
        df = df.withColumn(
            c,
            when(col(c).isNull() | (trim(col(c)) == ""), None).otherwise(col(c))
        )
    return df


def non_values_to_not_informed(df: DataFrame, columns: list[str], target_values: list[str], not_informed_str="NAO INFORMADO") -> DataFrame:
    """
    Converte para 'NAO INFORMADO':
      - valores contidos em 'targets'
      - strings com valor "0"
      - strings vazias ("")
      - strings contendo apenas espaços ("   ", "\t", etc. após trim)
    
    Aplica somente em colunas string.
    Se 'columns' for None, aplica a todas as colunas string do DataFrame.
    Se 'columns' contiver colunas não-string, elas serão ignoradas.
    """
    # Define colunas alvo
    if columns is None:
        target_cols = [c for c, t in df.dtypes if t == "string"]
    else:
        # Usa apenas colunas que existem e são string
        string_cols = {c for c, t in df.dtypes if t == "string"}
        target_cols = [c for c in columns if c in string_cols]

    print("")
    # Aplica a normalização usando a lista de valores 'target_values' como valores exatos a serem substituídos caso encontrados nas colunas
    for c in target_cols:
        for target_value in target_values:
            df = df.withColumn(
                c,
                when((col(c) == target_value), lit(not_informed_str)).otherwise(col(c))
            )
    return df
    
def eliminate_row_if_null_or_blank(df: DataFrame, columns_to_check: list[str]) -> DataFrame:
    """Elimina linhas onde as colunas listadas no parâmetro 'columns_to_check' for nula ou vazia.
    df: objeto DataFrame alvo do tratamento
    columns_to_check: lista de colunas a serem verificadas

    returns: objeto DataFrame com linhas eliminadas ou o mesmo DataFrame caso não haja linhas a serem eliminadas

    """
    for column_name in columns_to_check:
        df = df.filter(~(col(column_name).isNull()) & ~(col(column_name) == "") & ~(col(column_name) == " "))
    return df



def recommend_salt_simple(df, col):
    """
    Recomenda um valor de salt (número de partições extras) para balancear dados em joins ou writes, 
    baseado na distribuição dos valores de uma coluna.

    O algoritmo:
      - Conta o número de linhas para cada valor distinto da coluna.
      - Calcula o máximo e a mediana dessas contagens.
      - Avalia o grau de skew (desequilíbrio) pela razão max/mediana.
      - Retorna:
          1   se a distribuição for boa (pouco skew)
          4   para skew leve
          8   para skew moderado
          16  para skew forte

    
    df: DataFrame de entrada.
    col: Nome da coluna a ser avaliada.

    Retorna:
        int: Valor recomendado de salt.
    """
    # conta linhas por valor
    counts = df.groupBy(col).count()

    # métricas simples
    stats = counts.agg(
        max("count").alias("max_count"),
        percentile_approx("count", 0.5).alias("median_count")
    ).collect()[0]

    max_c = stats["max_count"]
    med_c = stats["median_count"]

    if med_c == 0:
        return 1  # nada a fazer

    ratio = max_c / med_c

    # regra simples
    if ratio < 1.5:
        salt = 1        # distribuição boa
    elif ratio < 2.5:
        salt = 4        # skew leve
    elif ratio < 4:
        salt = 8        # skew moderado (seu caso)
    else:
        salt = 16       # skew forte

    return salt


def _estimate_row_size_parquet(df, sample_rows=200_000, tmp_dir="dbfs:/tmp/row_size_estimate"):
    """
    Estima o tamanho médio (em bytes) de uma linha de um DataFrame Spark ao ser gravada em formato Parquet.

    Parâmetros:
        df (DataFrame): DataFrame Spark de entrada.
        sample_rows (int, opcional): Número de linhas a serem amostradas para a estimativa. Padrão: 200_000.
        tmp_dir (str, opcional): Diretório temporário no DBFS para salvar o arquivo Parquet amostrado. Padrão: "dbfs:/tmp/row_size_estimate".

    Retorna:
        float: Tamanho médio estimado de uma linha em bytes. Retorna 0.0 se não houver linhas.

    Observações:
        - Utiliza compressão 'snappy' ao gravar o Parquet.
        - Remove os arquivos temporários após a estimativa.
        - Usa builtins.sum para evitar conflitos com funções Spark.
    """
    tmp_path = f"{tmp_dir}/est_{uuid.uuid4().hex}"
    (df.limit(int(sample_rows))
       .coalesce(1)
       .write.mode("overwrite")
       .option("compression", "snappy")
       .parquet(tmp_path))

    n = spark.read.parquet(tmp_path).count()

    files = dbutils.fs.ls(tmp_path)
    # USE o builtins.sum (não o sum do Spark)
    total_bytes = builtins.sum([f.size for f in files if f.size is not None])

    dbutils.fs.rm(tmp_path, recurse=True)
    return (total_bytes / n) if n > 0 else 0.0


def recommend_partitions(df, desired_file_mb=256, sample_rows=200_000):
    """
    Recomenda o número de partições para gravar um DataFrame Spark, visando arquivos Parquet próximos ao tamanho desejado.

    Parâmetros:
        df (DataFrame): DataFrame Spark de entrada.
        desired_file_mb (int, opcional): Tamanho desejado de cada arquivo Parquet em megabytes. Padrão: 256.
        sample_rows (int, opcional): Número de linhas a serem amostradas para estimar o tamanho médio da linha. Padrão: 200_000.

    Retorna:
        dict: Dicionário com as seguintes chaves:
            - avg_row_bytes (float): Tamanho médio estimado de uma linha em bytes.
            - total_rows (int): Número total de linhas do DataFrame.
            - total_mb_estimated (float): Tamanho total estimado dos dados em megabytes.
            - desired_file_mb (int): Tamanho desejado de cada arquivo em megabytes.
            - recommended_partitions (int): Número recomendado de partições para atingir o tamanho de arquivo desejado.
    """
    avg_row_bytes = _estimate_row_size_parquet(df, sample_rows=sample_rows)
    total_rows = df.count()
    desired_bytes = desired_file_mb * 1024 * 1024
    total_bytes = total_rows * avg_row_bytes

    # Também use builtins.max para evitar conflitos se tiver importado max do Spark
    num_partitions = builtins.max(1, int(total_bytes / desired_bytes)) if desired_bytes > 0 else 1

    return {
        "avg_row_bytes": float(avg_row_bytes),
        "total_rows": int(total_rows),
        "total_mb_estimated": float(total_bytes / (1024*1024)),
        "desired_file_mb": int(desired_file_mb),
        "recommended_partitions": int(num_partitions),
    }




In [0]:
# Cnae - Tratamento e filtros

# Abrindo o arquivo dos Cnaes na Bronze
cnaes_df = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Cnaes")

# Recuperando as colunas
columns = cnaes_df.columns

# -- Sequência de tratamentos --

# strip/trim
cnaes_df = strip_df(cnaes_df, ["descricao"])

# Colunas em branco ou somente com espaços terão valor nulificado
cnaes_df = normalize_blanks_to_null(cnaes_df, ["descricao"])

# LPAD dos cnaes
cnaes_df = cnaes_df.withColumn("codigo", lpad(col('codigo'), 7, "0") )

# Dropando duplicados
cnaes_df = cnaes_df.dropDuplicates()

display(cnaes_df)

# Salvando no storage da silver
(
    cnaes_df.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/cnpj/Cnaes_tratado")
)


In [0]:
# -- Empresas - Tratamento e filtros --

# Abrindo o arquivo dos Empresas na Bronze
empresas_df = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Empresas")

# Recuperando as colunas
columns = empresas_df.columns

# -- Sequência de tratamentos --

# strip/trim
target_columns = ["RAZAO_SOCIAL_NOME_EMPRESARIAL"]
empresas_df = strip_df(empresas_df, target_columns)

# Colunas em branco ou somente com espaços terão valor nulificado
empresas_df = normalize_blanks_to_null(empresas_df, target_columns)

# LPAD para o cnpj
empresas_df = empresas_df.withColumn("CNPJ_BASICO", lpad(empresas_df.CNPJ_BASICO, 8, "0"))

# Colocando a pontuação correta para o decimal(substituindo ',' por '.') e convertendo para decimal
empresas_df = empresas_df.withColumn("CAPITAL_SOCIAL_DA_EMPRESA", regexp_replace(empresas_df.CAPITAL_SOCIAL_DA_EMPRESA, ",", ".")) \
                .withColumn("CAPITAL_SOCIAL_DA_EMPRESA", col("CAPITAL_SOCIAL_DA_EMPRESA").cast(DecimalType(18, 2)))

# Substituindo valores nulos na coluna 'ENTE_FEDERATIVO_RESPONSAVEL' por "NAO INFORMADO"
empresas_df = empresas_df.withColumn("ENTE_FEDERATIVO_RESPONSAVEL", when(col("ENTE_FEDERATIVO_RESPONSAVEL").isNull(), "NAO INFORMADO").otherwise(col("ENTE_FEDERATIVO_RESPONSAVEL")))

# Substituindo valores nulos na coluna 'PORTE_DA_EMPRESA' por "NAO INFORMADO"
empresas_df = empresas_df.withColumn("PORTE_DA_EMPRESA", when(col("PORTE_DA_EMPRESA").isNull(), "NAO INFORMADO").otherwise(col("PORTE_DA_EMPRESA")))

# Dropando duplicados
empresas_df = empresas_df.dropDuplicates()

display(empresas_df)

# Salvando os dados no storage da silver
(
    empresas_df.write
    .mode("overwrite")
    .save(f"{path_storage_silver}/cnpj/Empresas_tratado")
)


In [0]:
# -- Estabelecimentos - Tratamento e filtros --

# Abrindo o arquivo dos Estabelecimentos na Bronze
estabelecimentos_df = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Estabelecimentos")

# # Recuperando as colunas
# columns = estabelecimentos_df.columns


# -- Sequência de tratamentos --

# Eliminando linhas onde a coluna CNPJ_BASICO é nulo, string vazia ou contém somente espaços em branco
# estabelecimentos_df = estabelecimentos_df.filter(~(col("CNPJ_BASICO").isNull()) & ~(col("CNPJ_BASICO") == "") & ~(col("CNPJ_BASICO") == " "))
estabelecimentos_df = eliminate_row_if_null_or_blank(estabelecimentos_df, ["CNPJ_BASICO"])

# Eliminando linhas com CNPJ_BASICO inválido
estabelecimentos_df = estabelecimentos_df.filter(~((estabelecimentos_df.CNPJ_BASICO == "00000000") | (estabelecimentos_df.CNPJ_BASICO == "") | (estabelecimentos_df.CNPJ_BASICO.isNull()) ))

# Removendo colunas conforme acordado como José
columns_to_remove = [
    "DDD_1", "TELEFONE_1", "DDD_2", "TELEFONE_2", "DDD_FAX", "FAX", "CORREIO_ELETRONICO", "TIPO_LOGRADOURO", "LOGRADOURO", "NUMERO", "COMPLEMENTO", "CEP",
    "SITUACAO_ESPECIAL", "DATA_SITUACAO_ESPECIAL", 'ingestion_dt'
]
estabelecimentos_df = estabelecimentos_df.drop(*columns_to_remove)

# strip/trim
target_columns = ["NOME_FANTASIA", "NOME_CIDADE_EXTERIOR", "PAIS", "BAIRRO", "UF", "MUNICIPIO", 'CNAE_FISCAL_PRINCIPAL', 'CNAE_FISCAL_SECUNDARIA']
estabelecimentos_df = strip_df(estabelecimentos_df, target_columns)

# Colunas em branco ou somente com espaços terão valor nulificado
estabelecimentos_df = normalize_blanks_to_null(estabelecimentos_df, target_columns)

# LPAD na coluna CNPJ_BASICO
estabelecimentos_df = estabelecimentos_df.withColumn("CNPJ_BASICO", lpad(estabelecimentos_df.CNPJ_BASICO, 8, "0"))

# LPAD na coluna LPDAD CNPJ_ORDEM
estabelecimentos_df = estabelecimentos_df.withColumn("CNPJ_ORDEM", lpad(estabelecimentos_df.CNPJ_ORDEM, 4, "0"))

# LPAD na coluna CNPJ_DV
estabelecimentos_df = estabelecimentos_df.withColumn("CNPJ_DV", lpad(estabelecimentos_df.CNPJ_DV, 2, "0"))

# Colunas com valor nulo/NULL que receberão "NAO INFORMADO"
null_columns_target = ["NOME_FANTASIA", "SITUACAO_CADASTRAL", "MOTIVO_SITUACAO_CADASTRAL", "NOME_CIDADE_EXTERIOR", "PAIS", "BAIRRO", "UF", "MUNICIPIO"]
estabelecimentos_df = estabelecimentos_df.fillna("NAO INFORMADO", null_columns_target)

# Convertendo colunas string que armazenam data usando a função 'cast_dates' 
target_columns = ["DATA_SITUACAO_CADASTRAL", "DT_INICIO_ATIVIDADE"]
estabelecimentos_df = cast_dates(estabelecimentos_df, target_columns)

# Tratando o CNAE_FISCAL_PRINCIPAL usando lpad 7
estabelecimentos_df = estabelecimentos_df.withColumn("CNAE_FISCAL_PRINCIPAL", lpad(col("CNAE_FISCAL_PRINCIPAL"), 7, "0"))

# -- Tratamento da coluna CNAE_FISCAL_SECUNDARIO --
# Aplicando a função 'explode' na coluna CNAE_FISCAL_SECUNDARIO e pegando uma linha por item da lista que está em formato string separado por ','
estabelecimentos_df = estabelecimentos_df.withColumn("CNAE_FISCAL_SECUNDARIA_EXP", explode(split(col("CNAE_FISCAL_SECUNDARIA"), ",")))

# Aplicando a função "lpad" para cada um dos CNAE_FISCAL_SECUNDARIO 
estabelecimentos_df = estabelecimentos_df.withColumn("CNAE_FISCAL_SECUNDARIA_EXP", lpad(col("CNAE_FISCAL_SECUNDARIA_EXP"), 7, "0")) 
# print(f"DEBUG 1 {str(estabelecimentos_df.columns)}")

# Colunas do group_by para agrupar os dados novamente e obter uma lista dos CNAE_FISCAL_SECUNDARIO
group_columns =['CNPJ_BASICO', 'CNPJ_ORDEM', 'CNPJ_DV', 'IDENTIFICADOR_MATRIZ_FILIAL', 'NOME_FANTASIA', 'SITUACAO_CADASTRAL', 'DATA_SITUACAO_CADASTRAL', 'MOTIVO_SITUACAO_CADASTRAL', 'NOME_CIDADE_EXTERIOR', 'PAIS', 'DT_INICIO_ATIVIDADE', 'CNAE_FISCAL_PRINCIPAL', 'BAIRRO', 'UF', 'MUNICIPIO', 'origin_path_name', 'anomes']

# Agrupando para obter novamente uma lista da coluna CNAE_FISCAL_SECUNDARIA_EXP, que já tem o tratamento feito
estabelecimentos_df = estabelecimentos_df.groupBy(*group_columns).agg(collect_list("CNAE_FISCAL_SECUNDARIA_EXP").alias("CNAE_FISCAL_SECUNDARIA_BACK"))

# Concatenando CNAE_FISCAL_SECUNDARIA_BACK usando concat_ws para voltar o formato original do dado mas com o conteúdo tratado
estabelecimentos_df = estabelecimentos_df.withColumn("CNAE_FISCAL_SECUNDARIA", concat_ws(",", col("CNAE_FISCAL_SECUNDARIA_BACK"))) \
                        .drop("CNAE_FISCAL_SECUNDARIA_BACK")

# Partição sintética com os dois primeiros dígitos de CNPJ_BASICO
estabelecimentos_df = estabelecimentos_df.withColumn("CNPJ_BASICO_2D", substring(col("CNPJ_BASICO"), 1, 2))

# -- Flags úteis para estabelecimento --

# Adicionando a 'flag' 'IS_MATRIZ'(True se for matriz. Senão, False)
estabelecimentos_df = estabelecimentos_df.withColumn("IS_MATRIZ", when(col("IDENTIFICADOR_MATRIZ_FILIAL") == "1", True).otherwise(False))

# Adicionando a 'flag' 'IS_ATIVA' (True se o estabelecimento estiver ativo. Senão, False)
estabelecimentos_df = estabelecimentos_df.withColumn("IS_ATIVA", when(col("SITUACAO_CADASTRAL") == "02", True).otherwise(False))

# -- Tratamento das filiais --
# Contando as filiais por CNPJ_BASICO usando a coluna 'IS_MATRIZ' com valor False
filiais_df = estabelecimentos_df.filter(col("IS_MATRIZ") == False).groupBy("CNPJ_BASICO").count()
filiais_df = filiais_df.withColumnRenamed("count", "QTDE_FILIAIS")

# Adicionando a coluna QTDE_FILIAIS em estabelecimentos_df usando filiais_df e atribuindo valor 0 quando for uma filial
estabelecimentos_df = estabelecimentos_df.join(filiais_df, on="CNPJ_BASICO", how="left").fillna(0, subset=['QTDE_FILIAIS'])

# display(estabelecimentos_df)

# -- Salvando os dados no storage da "Silver" --

# Calculando o número de partições
recommendations = recommend_partitions(estabelecimentos_df)
print(f"RECOMMENDATIONS: {str(recommendations)}")
partitions_number = builtins.max(1, int(recommendations['recommended_partitions']))

# Calculando o 'salt' para a distribuição de partições/buckets
# salt_buckets = recommend_salt_simple(estabelecimentos_df, "CNPJ_BASICO_2D")
salt_buckets = builtins.int(recommend_salt_simple(estabelecimentos_df, "CNPJ_BASICO_2D"))
salt_buckets = builtins.max(1, salt_buckets)

# Criando a coluna com o salt para o reparticionamento
# estabelecimentos_df = estabelecimentos_df.withColumn("salt", (xxhash64(col("CNPJ_BASICO_2D")) % lit(salt_buckets)).cast("int"))
if salt_buckets == 1:
    estabelecimentos_df = estabelecimentos_df.withColumn("salt", lit(0).cast("int"))
else:
    estabelecimentos_df = estabelecimentos_df.withColumn(
        "salt",
        (xxhash64(col("CNPJ_BASICO_2D")) % lit(salt_buckets)).cast("int")
    )

# Reparticionando o dataframe usando a recomendação de reparticionamento
estabelecimentos_df = estabelecimentos_df.repartition(partitions_number, col("salt"))

# Gravando os dados no storage da "Silver"
(
    estabelecimentos_df.write
    .format("delta")
    .mode("overwrite")
    .partitionBy("anomes", "salt")
    .option("overwriteSchema", "true")
    .save(f"{path_storage_silver}/cnpj/Estabelecimentos_tratado")
)
 
# Sugestão modo overwriteSchema simples
# (
#     estabelecimentos_df.write
#     .format("delta")
#     .mode("overwrite")
#     .partitionBy("anomes", "CNPJ_BASICO_2D")
#     .option("overwriteSchema", "true").save(f"{path_storage_silver}/cnpj/Estabelecimentos_tratado")
# )


In [0]:
# # Contando as filiais por CNPJ_BASICO quando CNPJ_ORDEM não terminmar com '1' usando a coluna 'IS_MATRIZ' com valor False
# filiais_df = estabelecimentos_df.filter(col("IS_MATRIZ") == False).groupBy("CNPJ_BASICO").count()
# filiais_df = filiais_df.withColumnRenamed("count", "QTDE_FILIAIS")

# # Adicionando a coluna QTDE_FILIAIS em estabelecimentos_df usando filiais_df e atribuindo valor 0 quando for uma filial
# estabelecimentos_df = estabelecimentos_df.join(filiais_df, on="CNPJ_BASICO", how="left").fillna(0, subset="QTDE_FILIAIS")

# display(estabelecimentos_df)

# Contando o campo CNPJ_BASICO_2D para verificar a distribuição dos dois primeiros dígitos de CNPJ_BASICO
display(estabelecimentos_df.groupBy("CNPJ_BASICO_2D").count())



In [0]:
# Motivos - Tratamento e filtros --
df_motivos = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Motivos")

# Recuperando as colunas
columns = df_motivos.columns

# -- Sequência de tratamentos --

# strip/trim
df_motivos = strip_df(df_motivos, ["descricao"])

# Colunas em branco ou somente com espaços terão valor nulificado
df_motivos = normalize_blanks_to_null(df_motivos, ["descricao"])

display(df_motivos)

# Salvando os dados no storage da "Silver"
(
    df_motivos.write
    .format("delta")
    .mode("overwrite")
    .save(f"{path_storage_silver}/cnpj/Motivos_tratado")
)



In [0]:
# Municipios - Tratamento e filtros --
df_municipios = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Municipios")

# Recuperando as colunas
columns = df_municipios.columns

# -- Sequência de tratamentos --
# 1. UTF-8
# df_municipios = latin1_to_utf8(df_municipios, ["descricao"])

# 2. strip/trim
df_municipios = strip_df(df_municipios, ["descricao"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_municipios = normalize_blanks_to_null(df_municipios, ["descricao"])
 
# Tratando o CNAE_FISCAL_PRIMARIO usando lpad
df_municipios = df_municipios.withColumn("codigo", lpad(col("codigo"), 4, "0"))

display(df_municipios)

# Salvando os dados no storage da "Silver"
(
    df_motivos.write
    .format("delta")
    .mode("overwrite")
    .save(f"{path_storage_silver}/cnpj/Municipios_tratado")
)


In [0]:
# Naturezas - Tratamentos e Filtro --

#1. Lendo os arquivos delta de naturezas
df_naturezas = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Naturezas")

colunas_tratamento = ["DESCRICAO","CODIGO"]

# 2. strip/trim
df_naturezas = strip_df(df_naturezas, colunas_tratamento)

# 3. Remove acentos e caracteres especiais
df_naturezas = remove_accents(df_naturezas, colunas_tratamento)

# 4. Normaliza blanks para null
df_naturezas = normalize_blanks_to_null(df_naturezas, colunas_tratamento)

# 5. LPAD do código da Natureza juridica para 4 digitos
df_naturezas = df_naturezas.withColumn("CODIGO", lpad(col('CODIGO'), 4, "0") )

# 6. Convertendo a descrição para maiúsculo
df_naturezas = df_naturezas.withColumn(
    "DESCRICAO",
    upper(col("DESCRICAO"))
)

# 7. Aplicando filtros
df_naturezas = df_naturezas.filter(
    col("CODIGO").isNotNull() &
    col("DESCRICAO").isNotNull() &
    (length(col("CODIGO")) == 4)
)

#8. dropando duplicados 
df_naturezas = df_naturezas.dropDuplicates(["CODIGO"])

# 9. Dropando ingestion_dt e anomes
df_naturezas = df_naturezas.drop('ingestion_dt','anomes',"origin_path_name")

# Resultado final
display(df_naturezas)

try:   
    # Salvando no storage da silver
    (
    df_naturezas.write
        .mode("overwrite")
        .format("delta")
        .save(f"{path_storage_silver}/cnpj/Naturezas_Tratado")
    )
except Exception as e:
    print(f"Falha ao salvar storage na Silver: {e}")

try:   
   df_naturezas.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.Naturezas_Tratado")
except Exception as e:
    print(f"Falha ao salvar a delta table: {e}")

In [0]:
# Paises - Tratamentos e Filtro --

#1. Lendo os arquivos delta de Paises
df_Paises = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Paises")

colunas_tratamento = ["DESCRICAO","CODIGO"]

# 2. strip/trim
df_Paises = strip_df(df_Paises, colunas_tratamento)

# 3. Remove acentos e caracteres especiais
df_Paises = remove_accents(df_Paises, colunas_tratamento)

# 4. Normaliza blanks para null
df_Paises = normalize_blanks_to_null(df_Paises, colunas_tratamento)

# 5. LPAD da Natureza juridica para 4 digitos
df_Paises = df_Paises.withColumn("CODIGO", lpad(col('CODIGO'), 4, '0'))

# 6. Convertendo a descrição para maiúsculo
df_Paises = df_Paises.withColumn(
    "DESCRICAO",
    upper(col("DESCRICAO"))
)

# 7. Aplicando filtros
df_Paises = df_Paises.filter(
    col("CODIGO").isNotNull() &
    col("DESCRICAO").isNotNull() &
    (length(col("CODIGO")) == 4)
)

#8. dropando duplicados 
df_Paises = df_Paises.dropDuplicates(["CODIGO"])

# 9. Dropando ingestion_dt e anomes
df_Paises = df_Paises.drop('ingestion_dt','anomes','origin_path_name')

# Resultado final
display(df_Paises)

try:   
    # Salvando no storage da silver
    (
    df_Paises.write
        .mode("overwrite")
        .format("delta")
        .save(f"{path_storage_silver}/cnpj/Paises_Tratado")
    )
except Exception as e:
    print(f"Falha ao salvar storage na Silver: {e}")

try:
    df_Paises.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable("silver.Paises_Tratado")

except Exception as e:
    print(f"Falha ao salvar a delta table: {e}")



In [0]:
# Qualificacoes - Tratamentos e Filtro --

#1. Lendo os arquivos delta de Qualificacoes
df_qualificacoes = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Qualificacoes")

colunas_tratamento = ["DESCRICAO", "CODIGO"]

# 2. strip/trim
df_qualificacoes = strip_df(df_qualificacoes, colunas_tratamento)

# 3. Remove acentos e caracteres especiais
df_qualificacoes = remove_accents(df_qualificacoes, colunas_tratamento)

# 4. Normaliza blanks para null
df_qualificacoes = normalize_blanks_to_null(df_qualificacoes, colunas_tratamento)

# 5. LPAD da Natureza juridica para 4 digitos
df_qualificacoes = df_qualificacoes.withColumn("CODIGO", lpad(col('CODIGO'), 4, "0") )

# 6. Convertendo a descrição para maiúsculo
df_qualificacoes = df_qualificacoes.withColumn(
    "DESCRICAO",
    upper(col("DESCRICAO"))
)

# 7. Removendo linhas duplicadas 
df_qualificacoes = df_qualificacoes.filter(
    col("CODIGO").isNotNull() &
    col("DESCRICAO").isNotNull() &
    (length(col("CODIGO")) == 4)
)

#8. dropando duplicados 
df_qualificacoes = df_qualificacoes.dropDuplicates(["CODIGO"])

# 9. Dropando ingestion_dt e anomes
df_qualificacoes = df_qualificacoes.drop('ingestion_dt','anomes','origin_path_name')

# Resultado final
display(df_qualificacoes)

try:
    # Salvando no storage da silver
    (
        df_qualificacoes.write
        .mode("overwrite")
        .format("delta")
        .save(f"{path_storage_silver}/cnpj/Qualificacoes_Tratado")
    )
except Exception as e:
    print(f"Falha ao salvar storage na Silver: {e}")

try:
    # Salvando em delta table
    (
    df_qualificacoes.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.Qualificacoes_Tratado")
    )
except Exception as e:
    print(f"Falha ao salvar em delta table: {e}")     


In [0]:
# Simples - Tratamentos e Filtro --

# 1. Lendo os arquivos delta de simples
df_simples = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Simples")

# 2. colunas para tratamento de string
colunas_tratamento = ["CNPJ_BASICO","OPCAO_PELO_SIMPLES","OPCAO_PELO_MEI","DATA_DE_OPCAO_PELO_SIMPLES","DATA_DE_EXCLUSAO_DO_SIMPLES","DATA_DE_OPCAO_PELO_MEI","DATA_DE_EXCLUSAO_DO_MEI"]

# 3. strip/trim
df_simples = strip_df(df_simples, colunas_tratamento)

# 4. Remove acentos e caracteres especiais
df_simples = remove_accents(df_simples, colunas_tratamento)

# 5. Normaliza blanks para null
df_simples = normalize_blanks_to_null(df_simples, colunas_tratamento)

# 6. colunas para tratamento de datas
colunas_tratamento_datas = ["DATA_DE_OPCAO_PELO_SIMPLES","DATA_DE_EXCLUSAO_DO_SIMPLES","DATA_DE_OPCAO_PELO_MEI","DATA_DE_EXCLUSAO_DO_MEI"]

# 7. função para tratamentos de data
df_simples = cast_dates(df_simples, colunas_tratamento_datas)

# 8. LPAD do cnpj_basico para 8 digitos
df_simples = df_simples.withColumn("CNPJ_BASICO", lpad(col('CNPJ_BASICO'), 8, "0") )

# 9. Padronizando colunas de opção pelo MEI
df_simples = df_simples.withColumn(
    "OPCAO_PELO_SIMPLES",
    upper(col("OPCAO_PELO_SIMPLES"))
).withColumn(
    "OPCAO_PELO_MEI",
    upper(col("OPCAO_PELO_MEI"))
)

# 10. Aplicando filtros 
df_simples = df_simples.filter(
    col("CNPJ_BASICO").isNotNull() &
    (length(col("CNPJ_BASICO")) == 8)
)
df_simples = df_simples.filter(
    col("OPCAO_PELO_SIMPLES").isin("S", "N") &
    col("OPCAO_PELO_MEI").isin("S", "N")
)

# 11. dropando duplicados 
df_simples = df_simples.dropDuplicates(["CNPJ_BASICO"])

# 12. Dropando ingestion_dt e anomes
df_simples = df_simples.drop('ingestion_dt','anomes','origin_path_name')

# Resultado final
display(df_simples)

try:
    # Salvando no storage da silver
    (
        df_simples.write
        .mode("overwrite")
        .format("delta")
        .save(f"{path_storage_silver}/cnpj/Simples_Tratado")
    )
except Exception as e:
    print(f"Falha ao salvar storage na Silver: {e}")

try:
    # Salvando em delta table
    (
    df_simples.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver.Simples_Tratado")
    )
except Exception as e:
    print(f"Falha ao salvar em delta table: {e}")     

In [0]:
# Socios - Tratamentos e filtros --

# 1. Lendo os arquivos delta de socios
df_socios = spark.read.format("delta").load(f"{path_storage_bronze}/cnpj/Socios")

colunas_tratamento = ["CNPJ_BASICO","NOME_DO_SOCIO","PAIS","QUALIFICACAO_DO_SOCIO","IDENTIFICADOR_DO_SOCIO","CPF/CNPJ_DO_SOCIO","FAIXA_ETARIA","DATA_DE_ENTRADA_NA_SOCIEDADE"]

# 2. strip/trim
df_socios = strip_df(df_socios, colunas_tratamento)

# 3. Remove acentos e caracteres especiais
df_socios = remove_accents(df_socios, colunas_tratamento)

# 4. Normaliza blanks para null
df_socios = normalize_blanks_to_null(df_socios, colunas_tratamento)

# 5. LPAD do cnpj básico
df_socios = df_socios.withColumn("CNPJ_BASICO", lpad(col('CNPJ_BASICO'), 8, "0") )

# 6. Tratamento de datas 
colunas_tratamento_data = ["DATA_DE_ENTRADA_NA_SOCIEDADE"]

df_socios = cast_dates(df_socios, colunas_tratamento_data) 

# 7. Remoção de colunas
df_socios = df_socios.drop("nome_do_representante_legal","qualificacao_do_representante_legal","representante_legal","anomes","ingestion_dt")

# 8. Transformando colunas nulas em (105 -> brasil)
df_socios = df_socios.withColumn(
    "PAIS",
    when(
        (col("PAIS").isNull()) |
        (trim(col("PAIS")) == "") |
        (col("PAIS") == "null"),
        "105"
    ).otherwise(col("PAIS"))
)

# 9. Normalizando os campos
df_socios = df_socios.withColumn("NOME_DO_SOCIO", upper(col("NOME_DO_SOCIO"))) \
                     .withColumn("QUALIFICACAO_DO_SOCIO", upper(col("QUALIFICACAO_DO_SOCIO"))) \
                     .withColumn("PAIS", upper(col("PAIS")))

 
# 10. Filtrando só os cnpjs que se enquandram nos requisitos 
df_socios = df_socios.filter(
    col("CNPJ_BASICO").isNotNull() &
    (length(col("CNPJ_BASICO")) == 8)
)

# 11. Removendo duplicados
df_socios = df_socios.dropDuplicates(["CNPJ_BASICO","IDENTIFICADOR_DO_SOCIO"])

# 12. Dropando ingestion_dt e anomes
df_socios = df_socios.drop('ingestion_dt','anomes','origin_path_name')

# Resultado final
display(df_socios)

try:
    # Salvando no storage da silver
    (
        df_socios.write
        .mode("overwrite")
        .format("delta")
        .save(f"{path_storage_silver}/cnpj/Socios_Tratado")
    )
except Exception as e:
    print(f"Falha ao salvar storage na Silver: {e}")

try:
    # Salvando em Delta Table
    df_socios.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable("silver.Socios_Tratado")

except Exception as e:
    print(f"Falha ao salvar delta table: {e}")